Implements ESLB value estimator for contextual bandit off-policy problem.

All estimators are described in
"Kuzborskij, I., Vernade, C., Gyorgy, A., & Szepesvári, C. (2021, March).
Confident off-policy evaluation and selection through self-normalized importance
weighting. In International Conference on Artificial Intelligence and Statistics
(pp. 640-648). PMLR.".
In the following we occassionally refer to the statements in the paper
(e.g. Theorem 1, Proposition 1).

class ESLB implements an Efron-Stein high probability bound for off-policy
evaluation (Theorem 1 and Algorithm 1).

In [ ]:
# Copyright 2021 DeepMind Technologies Limited.
#
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import logging
import math
import enum
import numpy as np

# Tools

In [ ]:
def sample_from_simplices_m_times(p, m):
  """Samples from each of n probability simplices for m times.

  Args:
    p: n-times-K matrix where each row describes a probability simplex
    m: number of times to sample

  Returns:
    n-times-m matrix of indices of simplex corners.
  """
  axis = 1
  r = np.expand_dims(np.random.rand(p.shape[1 - axis], m), axis=axis)
  p_ = np.expand_dims(p.cumsum(axis=axis), axis=2)
  return (np.repeat(p_, m, axis=2) > r).argmax(axis=1)

# Implementation of the ESLB estimator

In [ ]:
class ESLBBiasType(enum.Enum):
  """Bias control type of ESLB estimator.

  ESLBBiasType.MultOneHot = Multiplicative bias (only for `one-hot' rewards).
  ESLBBiasType.Bernstein = Bernstein bias (for rewards in [0,1], looser than above).
  """
  MultOneHot = "MultOneHot"
  Bernstein = "Bernstein"

  def __str__(self):
    return str(self.value)

class ESLB(object):
  """Implements a Semi-Empirical Efron-Stein bound for the SNIW (Self-normalized Importance Weighted estimator).

  Attributes:
    delta: Error probability in (0,1).
    n_iterations: Number of Monte-Carlo simulation iterations for approximating
      a multiplicative bias and a variance proxy.
    n_batch_size: Monte-Carlo simulation batch size.
    bias_type: type of bias control to use (see ESLBBiasType).
  """

  def __init__(
      self,
      delta: float,
      n_iterations: int,
      n_batch_size: int,
      bias_type: ESLBBiasType
  ):
    """Constructs an estimator.

    The estimate holds with probability 1-delta.

    Args:
      delta: delta: Error probability in (0,1) for a confidence interval.
      n_iterations: Monte-Carlo simulation iterations.
      n_batch_size: Monte-Carlo simulation batch size.
      bias_type: type of bias control to use.
    """
    self.delta = delta
    self.n_iterations = n_iterations
    self.n_batch_size = n_batch_size
    self.bias_type = bias_type

  def get_name(self):
    """Returns the long name of the estimator."""
    return "Semi-Empirical Efron-Stein bound for the Self-normalized Estimator"

  def get_abbrev(self):
    """Returns the short name of the estimator."""
    return "ESLB"

  def __call__(
      self,
      t_probs: np.ndarray,
      b_probs: np.ndarray,
      actions: np.ndarray,
      rewards: np.ndarray,
  ):
    """Computes Efron-Stein lower bound of Theorem 1 as described in Algorithm 1.

    Here n is a sample size, while K is a number actions.

    Args:
      t_probs: n-times-K matrix, where i-th row corresponds to π_t(. | X_i)
        (target probabilities under the target policy).
      b_probs: n-times-K matrix, where i-th row corresponds to π_b(. | X_i)
        (target probabilities under the behavior policy).
      actions: n-sized vector of actions.
      rewards: n-sized reward vector.

    Returns:
      A dictionary with 8 entries:
        lower_bound: Corresponds to the actual lower bound.
        estimate: Same as lower_bound (required by select_policy(...)).
        est_value: Empirical value.
        mult_bias: Multiplicative bias.
        concentration_of_contexts: Hoeffding term, concentration of contexts.
        var_proxy: Variance proxy.
        expected_variance_proxy: Estimated expected counterpart.
    """
    conf = math.log(2.0 / self.delta)
    n = len(actions)
    ix_1_n = np.arange(n)

    # Importance weights
    weights = t_probs[ix_1_n, actions] / b_probs[ix_1_n, actions]

    weights_cumsum = weights.cumsum()
    weights_cumsum = np.repeat(
        np.expand_dims(weights_cumsum, axis=1), self.n_batch_size, axis=1)
    weights_repeated = np.repeat(
        np.expand_dims(weights, axis=1), self.n_batch_size, axis=1)

    weight_table = t_probs / b_probs

    var_proxy_unsumed = np.zeros((n,))
    expected_var_proxy_unsumed = np.zeros((n,))
    loo_expected_recip_weights = 0.0

    are_rewards_binary = ((rewards==0) | (rewards==1)).all()
    if self.bias_type == ESLBBiasType.MultOneHot and not are_rewards_binary:
      raise Exception("""bias_type=ESLBBiasType.MultOneHot only supports one-hot rewards. 
Consider using bias_type=ESLBBiasType.Bernstein""")

    logging.debug(
        "ESLB:: Running Monte-Carlo estimation of the variance proxy and bias")
    logging.debug("ESLB:: iterations = %d, batch size = %d", self.n_iterations,
                  self.n_batch_size)

    for i in range(self.n_iterations):
      actions_sampled = sample_from_simplices_m_times(
          b_probs, self.n_batch_size)
      weights_sampled = weight_table[ix_1_n, actions_sampled.T].T
      weights_sampled_cumsum = weights_sampled[::-1, :].cumsum(axis=0)[::-1, :]
      # Hybrid sums: sums of empirical and sampled weights
      weights_hybrid_sums = np.copy(weights_cumsum)
      weights_hybrid_sums[:-1, :] += weights_sampled_cumsum[1:, :]

      # Computing variance proxy
      weights_hybrid_sums_replace_k = weights_hybrid_sums - weights_repeated + weights_sampled

      sn_weights = weights_repeated / weights_hybrid_sums
      sn_weights_prime = weights_sampled / weights_hybrid_sums_replace_k

      var_proxy_t = (sn_weights + sn_weights_prime)**2
      var_proxy_new_item = var_proxy_t.mean(axis=1)
      var_proxy_unsumed += (var_proxy_new_item - var_proxy_unsumed) / (i + 1)

      actions_sampled_for_expected_var = sample_from_simplices_m_times(
          b_probs, self.n_batch_size)
      weights_sampled_for_expected_var = weight_table[
          ix_1_n, actions_sampled_for_expected_var.T].T

      expected_var_proxy_new_item = (
          (weights_sampled_for_expected_var /
           weights_sampled_for_expected_var.sum(axis=0))**2).mean(axis=1)
      expected_var_proxy_unsumed += (expected_var_proxy_new_item -
                                     expected_var_proxy_unsumed) / (i + 1)


      if self.bias_type == ESLBBiasType.MultOneHot:
        # Computing bias (loo = leave-one-out)
        # Rewards are `one-hot'
        actions_sampled_for_bias = sample_from_simplices_m_times(
          b_probs, self.n_batch_size)
        weights_sampled_for_bias = weight_table[ix_1_n,
                                                actions_sampled_for_bias.T].T
        loo_sum_weights = np.outer(
          np.ones((n,)),
          np.sum(weights_sampled_for_bias, axis=0)
        ) - weights_sampled_for_bias
        loo_expected_recip_weights += (1 / np.min(loo_sum_weights, axis=0)).mean()

    var_proxy = var_proxy_unsumed.sum()
    expected_var_proxy = expected_var_proxy_unsumed.sum()

    if self.bias_type == ESLBBiasType.MultOneHot:
      loo_expected_recip_weights /= self.n_iterations
      eff_sample_size = 1.0 / loo_expected_recip_weights
      mult_bias = min(1.0, eff_sample_size / n)
      add_bias = 0
    elif self.bias_type == ESLBBiasType.Bernstein:
      # Computing Bernstein bias control (based on lower tail Bernstein's inequality)
      expected_sum_weights_sq = (t_probs**2 / b_probs).sum()
      bias_x = math.log(n) / 2
      mult_bias = 1 - math.sqrt(2 * expected_sum_weights_sq * bias_x) / n
      mult_bias = max(0, mult_bias)
      add_bias = math.exp(-bias_x)
      
    concentration = math.sqrt(
        2.0 * (var_proxy + expected_var_proxy) *
        (conf + 0.5 * math.log(1 + var_proxy / expected_var_proxy)))
    concentration_of_contexts = math.sqrt(conf / (2 * n))
    est_value = weights.dot(rewards) / weights.sum()
    lower_bound = mult_bias * (est_value
                               - concentration
                               - add_bias) - concentration_of_contexts

    return dict(
        estimate=max(0, lower_bound),
        lower_bound=max(0, lower_bound),
        est_value=est_value,
        concentration=concentration,
        mult_bias=mult_bias,
        concentration_of_contexts=concentration_of_contexts,
        var_proxy=var_proxy,
        expected_var_proxy=expected_var_proxy)

Minimal working example for using ``ESLB'' class, where the behavior and target policies are Softmax policies with the mass concentrated on one action (different for each policy).

In [ ]:
np.random.seed(1)

temp = 1  # temperature of Softmax policy
K = 5  # number of actions
delta_ = 0.05  # error probability of the lower bound

b_ix = 0  # mass on the 0-th action in the behavior policy
t_ix = 1  # mass on the 1-st action in the target policy

# Definition of the behavior Softmax policy
b_pot = np.zeros(K)
b_pot[b_ix] = 1
b_policy = np.exp(b_pot / temp)
b_policy /= b_policy.sum()

# Definition of the target Softmax policy
t_pot = np.zeros(K)
t_pot[t_ix] = 1
t_policy = np.exp(t_pot / temp)
t_policy /= t_policy.sum()

t_value = t_policy[t_ix]  # Value of the target policy

# Computation of the lower bound for increasing number of observations
for n_ in [100, 1000, 10000]:
  actions_ = np.random.choice(range(K), size=n_, p=b_policy)
  rewards_ = np.array(actions_ == t_ix, dtype=int)
  b_probs_ = np.repeat(np.expand_dims(b_policy, 0), n_, axis=0)
  t_probs_ = np.repeat(np.expand_dims(t_policy, 0), n_, axis=0)

  estimator = ESLB(delta=delta_, n_iterations=10, n_batch_size=1000, bias_type=ESLBBiasType.MultOneHot)
  results = estimator(
      t_probs=t_probs_, b_probs=b_probs_, actions=actions_, rewards=rewards_)

  print("------------------------------------------------------------------")
  print("behavior policy:\t (%s)" %
        ", ".join(map(lambda x: "%.2f" % x, b_policy)))
  print("target policy:\t\t (%s)" %
        ", ".join(map(lambda x: "%.2f" % x, t_policy)))
  print("sample size:\t\t %d" % n_)
  print("value(pi):\t\t %.3f" % t_value)
  print("ESLB:\t\t\t %.3f" % results["lower_bound"])
  print("hat{value}(pi):\t\t %.3f" % results["est_value"])
  print("conc.:\t\t\t %.3f" % results["concentration"])
  print("mult. bias:\t\t %.3f" % results["mult_bias"])
  print("conc. of contexts:\t %.3f" % results["concentration_of_contexts"])